Gathnex : https://gathnex.medium.com/how-to-create-your-own-llm-agent-from-scratch-a-step-by-step-guide-14b763e5b3b8

In [1]:
!pip install -q openai langchain py_expression_eval google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from openai import OpenAI
from langchain.utilities import SerpAPIWrapper, GoogleSearchAPIWrapper
from py_expression_eval import Parser
from langchain.tools import Tool
import re, time, os

client = OpenAI(api_key='Openai_api_key')
os.environ["GOOGLE_CSE_ID"] = "Google CSE ID"
os.environ["GOOGLE_API_KEY"] = "Google API Key"

In [3]:
search = GoogleSearchAPIWrapper()
goog = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run
    )

def search(str):
    return goog(str)

parser = Parser()
def calculator(str):
    return parser.parse(str).evaluate({})

In [4]:
System_prompt = """
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Calculator: Useful for when you need to answer questions about math. Use python code, eg: 2 + 2
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things

Option 1: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.

Option 2: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"

Begin!
"""

In [6]:
def Stream_agent(prompt):
    messages = [
        { "role": "system", "content": System_prompt },
        { "role": "user", "content": prompt },
    ]
    def extract_action_and_input(text):
          action_pattern = r"Action: (.+?)\n"
          input_pattern = r"Action Input: \"(.+?)\""
          action = re.findall(action_pattern, text)
          action_input = re.findall(input_pattern, text)
          return action, action_input
    while True:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
            max_tokens=1000,
            top_p=1,)
        response_text = response.choices[0].message.content
        print(response_text)
        time.sleep(30)
        action, action_input = extract_action_and_input(response_text)
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

In [7]:
Stream_agent("who is current CEO of OpenAI")

Thought: I need to find out who the current CEO of OpenAI is.
Action: Search
Action Input: "current CEO of OpenAI"
Observation:  2 days ago ... Mira Murati: Who is the new OpenAI interim CEO? Murati now heads the company ... Now, Musk can say he also used to employ OpenAI's current CEO. ^ Mok, Aaron (September 25, 2023). "OpenAI CEO Sam Altman says he worked so hard on building his first startup with his ex-boyfriend that he got scurvy". 2 days ago ... Chief technology officer Mira Murati appointed interim CEO to lead OpenAI; Sam Altman departs the company. CEO and co-founder: Sam Altman, former president of the startup accelerator Y Combinator (2015–2023); President and co-founder: Greg Brockman, former CTO, 3rd ... OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Jun 21, 2023 ... You ever watch Star Trek?” Sam Altman, the CEO who has become the most visible face of the current artificial-intellige